<a href="https://colab.research.google.com/github/Amirosimani/ReWOO-Gemini/blob/main/ReWOO_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --quiet datasets
# !pip install --quiet langchain langchain_community
# !pip install --quiet langchain_google_genai langchain_google_community
# !pip install --quiet wikipedia

In [3]:
from google.colab import userdata

In [5]:
# --- Configuration ---
MODEL = "gemini-1.5-flash"
MAX_ITERATIONS = 5
MAX_EXECUTION_TIME = 90

GENERATION_CONFIG = {
    "temperature": 0.8,
    "top_p": 0.95,
    "top_k": 20,
    "candidate_count": 1,
    "max_output_tokens": 8192,
    "stop_sequences": ["STOP!"],
}

SAFETY_SETTINGS = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


GEMINI_API_KEY = userdata.get('GEMINI')
GOOGLE_API_KEY = userdata.get('GOOGLE-API')
CSE_ID = userdata.get("CSE-ID")

## Data

[**StrategyQA**](https://paperswithcode.com/dataset/strategyqa) is a question answering benchmark where the required reasoning steps are implicit in the question, and should be inferred using a strategy. It includes 2,780 examples, each consisting of a strategy question, its decomposition, and evidence paragraphs. Questions in StrategyQA are short, topic-diverse, and cover a wide range of strategies.



In [8]:
from datasets import load_dataset

ds = load_dataset("ChilleD/StrategyQA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
# Access the training split
train_ds = ds["train"]

In [10]:
train_ds[0]

{'qid': '4fd64bb6ce5b78ab20b6',
 'term': 'Mixed martial arts',
 'description': 'full contact combat sport',
 'question': 'Is Mixed martial arts totally original from Roman Colosseum games?',
 'answer': False,
 'facts': 'Mixed Martial arts in the UFC takes place in an enclosed structure called The Octagon. The Roman Colosseum games were fought in enclosed arenas where combatants would fight until the last man was standing. Mixed martial arts contests are stopped when one of the combatants is incapacitated. The Roman Colosseum was performed in front of crowds that numbered in the tens of thousands. Over 56,000 people attended UFC 193.'}

# LLM

## Baseline

In [11]:
from google import genai
from google.genai.types import GenerateContentConfig

In [12]:
client = genai.Client(api_key=GEMINI_API_KEY)

In [20]:
def generate(prompt, model=MODEL, config=GENERATION_CONFIG):
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=GenerateContentConfig(**config)
    )
    return response

In [21]:
generate(prompt=train_ds[0]['question'])

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="No, mixed martial arts (MMA) is not totally original from Roman Colosseum games. While the Colosseum featured various forms of combat that included elements found in modern MMA, such as striking, grappling, and submissions,  there are crucial differences:\n\n* **Rules and Regulations:**  Colosseum combat was largely unregulated and brutal.  There were no weight classes, rounds, or referees to enforce rules.  The goal was often death or complete incapacitation, not a points-based victory as in modern MMA.\n\n* **Training and Specialization:** Modern MMA fighters undergo years of rigorous training in various martial arts disciplines. Colosseum fighters, while likely having some training, lacked the specialized and systematic training methods availab

## Native tool call

## ReAct Agent

In [1]:
import os
from typing import List, Dict, Union, Callable

import tiktoken
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.utilities import GoogleSearchAPIWrapper, WikipediaAPIWrapper
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish, LLMResult, BaseMessage
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain.tools import Tool

In [9]:
class ReActAgentExecutor:
    """
    A class to run the ReAct agent with specified configurations and tools.
    """

    def __init__(
        self,
        model: str = MODEL,
        generation_config: Dict = GENERATION_CONFIG,
        safety_settings: Dict = SAFETY_SETTINGS,
        max_iterations: int = MAX_ITERATIONS,
        max_execution_time: int = MAX_EXECUTION_TIME,
        google_api_key: str = GOOGLE_API_KEY,
        cse_id: str = CSE_ID,
    ):
        self.model = model
        self.generation_config = generation_config
        self.safety_settings = safety_settings
        self.max_iterations = max_iterations
        self.max_execution_time = max_execution_time
        self.google_api_key = google_api_key
        self.cse_id = cse_id
        self.llm = None
        self.tools = None
        self.agent = None
        self.agent_executor = None
        self.token_callback = None

        self._setup_llm()
        self._setup_tools()
        self._setup_agent()

    def _setup_llm(self):
      """Initializes the language model."""
      if not GEMINI_API_KEY or GEMINI_API_KEY == "your_gemini_api_key":
          raise ValueError("GEMINI_API_KEY must be set to a valid API key.")
      self.llm = ChatGoogleGenerativeAI(
          model=self.model,
          google_api_key=GEMINI_API_KEY,
          generation_config=self.generation_config,
          safety_settings=self.safety_settings,
      )

    def _setup_tools(self):
        """Sets up the tools for the agent."""
        search = GoogleSearchAPIWrapper(
            google_api_key=self.google_api_key, google_cse_id=self.cse_id
        )
        wikipedia = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)

        self.tools = [
            Tool(
                name="Google Search",
                func=search.run,
                description="Useful for finding information on current events, comparisons, or diverse perspectives.",
            ),
            Tool(
                name="Wikipedia",
                func=wikipedia.run,
                description="Useful for getting definitions and summaries of topics from Wikipedia.",
            ),
        ]

    def _setup_agent(self):
        """Sets up the ReAct agent and executor."""
        prompt = hub.pull("hwchase17/react")
        prompt_template = (
            prompt.template
            + """

        **Instructions:**
        1. Use Google Search to find information about current events, comparisons between topics, or to get diverse perspectives.
        2. Use Wikipedia to look up definitions, background information, or summaries of specific topics.
        3. Carefully read and understand the Observation from each tool before deciding on your next action.
        4. Do not repeat the same Action with the same Action Input multiple times in a row unless you have a strong reason to believe the result will be different. If you find yourself repeating, rethink your strategy in the Thought stage.

        Begin!
        """
        )
        prompt.template = prompt_template
        self.agent = create_react_agent(self.llm, self.tools, prompt)

        self.token_callback = TokenCountingCallbackHandler(self.model)
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=self.max_iterations,
            max_execution_time=self.max_execution_time,
            callbacks=[self.token_callback],
        )

    def run(self, input_data: Union[Dict, str]) -> Dict:
        """
        Runs the agent with the given input data.

        Args:
            input_data: Either a dictionary or a string representing the input for the agent.

        Returns:
            The output from the agent.
        """
        if isinstance(input_data, str):
            input_data = {"input": input_data}

        try:
            result = self.agent_executor.invoke(input_data)
            # Include token usage information in the result
            result["token_usage"] = {
                "total_tokens": self.token_callback.total_tokens,
                "prompt_tokens": self.token_callback.prompt_tokens,
                "completion_tokens": self.token_callback.completion_tokens,
            }
            self.token_callback.reset()  # Reset after each run
            return result
        except Exception as e:
            print(f"An error occurred: {e}")
            return {"error": str(e)}


class TokenCountingCallbackHandler(BaseCallbackHandler):
    """Callback handler for counting tokens used by the language model."""

    def __init__(self, model_name: str):
        self.model_name = model_name
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.encoding = tiktoken.get_encoding("cl100k_base")

    def on_llm_start(
        self, serialized: Dict[str, any], prompts: List[str], **kwargs
    ) -> None:
        """Collect prompt tokens when LLM starts."""
        for prompt in prompts:
            self.prompt_tokens += len(self.encoding.encode(prompt))

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        """Collect completion tokens when LLM finishes generating."""
        if response.generations:
            for generation_list in response.generations:
                for generation in generation_list:
                    if generation.text:
                        self.completion_tokens += len(
                            self.encoding.encode(generation.text)
                        )

    def on_agent_action(self, action: AgentAction, **kwargs) -> None:
        """Increment token count on agent action."""
        if action.log:
            self.total_tokens += len(self.encoding.encode(action.log))

    def on_agent_finish(self, finish: AgentFinish, **kwargs) -> None:
        """Increment token count on agent finish."""
        if finish.log:
            self.total_tokens += len(self.encoding.encode(finish.log))

    def on_chain_end(self, outputs, **kwargs) -> None:
        """Print the total tokens used when the chain finishes."""
        self.total_tokens += self.completion_tokens + self.prompt_tokens
        print(f"Prompt tokens: {self.prompt_tokens}")
        print(f"Completion tokens: {self.completion_tokens}")
        print(f"Total tokens used in this chain: {self.total_tokens}")

    def reset(self):
        """Reset the counters for the next chain run."""
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0

In [10]:
agent_executor = ReActAgentExecutor()
result = agent_executor.run(train_ds[0]["question"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Question: Is Mixed martial arts totally original from Roman Colosseum games?
Thought: I need to compare the characteristics of modern Mixed Martial Arts (MMA) and the combat sports of the Roman Colosseum to determine if there's a direct lineage.  Wikipedia would be a good starting point for defining both.
Action: Wikipedia
Action Input: "Mixed martial arts"Page: Mixed martial arts
Summary: Mixed martial arts (MMA) is a full-contact fighting sport based on striking and grappling, incorporating techniques from various combat sports from around the world.
In the early 20th century, various inter-stylistic contests took place throughout Japan and the countries of East Asia. At the same time, in Brazil there was a phenomenon called vale tudo, which became known for unrestricted fights between various styles such as judo, Brazilian jiu-jitsu, catch wrestling, luta livre, Muay Thai and capoeira. An early high-profile mixed bout was Kimura vs. Gracie in 

In [11]:
result["token_usage"]["total_tokens"]

416

# TODO:

[x] add baseline

[] add calc (optional)

[x] callback/count tokens for react

[] ReWOO